In [ ]:
# -*- coding: utf-8 -*-
"""
Модуль для проведения K-Fold Cross-Validation с расчетом доверительных интервалов для модели YOLOv8n.

Содержит функции для:
1. Загрузки и подготовки данных
2. Разделения данных на K фолдов
3. Обучения модели на каждом фолде
4. Анализа и визуализации результатов
"""

import os
import shutil
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from ultralytics import YOLO


class YOLOv8CrossValidation:
    """Класс для проведения кросс-валидации модели YOLOv8."""

    def __init__(self, base_path, num_folds=5, model_name="yolov8n.pt", epochs=50, imgsz=640, batch=32, device=0):
        """
        Инициализация параметров кросс-валидации.

        Args:
            base_path (str): Путь к директории с данными
            num_folds (int): Количество фолдов (по умолчанию 5)
            model_name (str): Название модели YOLO (по умолчанию "yolov8n.pt")
            epochs (int): Количество эпох обучения (по умолчанию 50)
            imgsz (int): Размер изображения (по умолчанию 640)
            batch (int): Размер батча (по умолчанию 32)
            device (int): ID GPU устройства (по умолчанию 0)
        """
        self.base_path = base_path
        self.num_folds = num_folds
        self.model_name = model_name
        self.epochs = epochs
        self.imgsz = imgsz
        self.batch = batch
        self.device = device

        self.class_names = [
            'city', 'course_period', 'course_topic', 'hours',
            'name', 'organization', 'registration_number', 'year'
        ]
        self.num_classes = len(self.class_names)

        self.all_metrics = []
        self.class_metrics = {name: [] for name in self.class_names}

        # Создаем директорию для сохранения результатов
        os.makedirs(os.path.join(self.base_path, "runs"), exist_ok=True)

    def collect_all_images(self, root_path):
        """
        Собирает все изображения и соответствующие метки.

        Args:
            root_path (str): Путь к корневой директории с данными

        Returns:
            list: Список словарей с информацией об изображениях и метках
        """
        image_files = []
        for root, _, files in os.walk(root_path):
            if "images" in root.lower():
                label_dir = root.replace("images", "labels")
                if not os.path.exists(label_dir):
                    print(f"Предупреждение: папка с метками не найдена {label_dir}")
                    continue

                for file in files:
                    if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                        name = os.path.splitext(file)[0]
                        label_path = os.path.join(label_dir, name + '.txt')
                        if os.path.exists(label_path):
                            image_files.append({
                                'name': name,
                                'image_path': os.path.join(root, file),
                                'label_path': label_path
                            })
                        else:
                            print(f"Предупреждение: метка не найдена для {file}")
        return image_files

    def prepare_kfold_data(self):
        """Подготовка данных для K-Fold кросс-валидации."""
        all_images = self.collect_all_images(self.base_path)
        if not all_images:
            raise ValueError("Не найдено ни одного изображения с метками!")

        print(f"Найдено {len(all_images)} изображений с метками")

        # Создаем KFold разделитель
        kf = KFold(n_splits=self.num_folds, shuffle=True, random_state=42)

        # Создаем директории для фолдов
        for i in range(self.num_folds):
            fold_dir = os.path.join(self.base_path, f"fold_{i+1}")
            for subset in ['train', 'valid']:
                os.makedirs(os.path.join(fold_dir, subset, "images"), exist_ok=True)
                os.makedirs(os.path.join(fold_dir, subset, "labels"), exist_ok=True)

        # Разделяем данные на фолды
        for fold, (train_idx, val_idx) in enumerate(kf.split(all_images), 1):
            self._process_fold(fold, all_images, train_idx, val_idx)

        print("\nРазделение на фолды успешно завершено!")

    def _process_fold(self, fold, all_images, train_idx, val_idx):
        """
        Обработка одного фолда: копирование данных и создание YAML-файла.

        Args:
            fold (int): Номер фолда
            all_images (list): Список всех изображений
            train_idx (array): Индексы тренировочных данных
            val_idx (array): Индексы валидационных данных
        """
        fold_dir = os.path.join(self.base_path, f"fold_{fold}")
        print(f"\nОбработка фолда {fold}...")

        # Копируем тренировочные данные
        for idx in train_idx:
            self._copy_image_and_label(all_images[idx], fold_dir, "train")

        # Копируем валидационные данные
        for idx in val_idx:
            self._copy_image_and_label(all_images[idx], fold_dir, "valid")

        # Проверка количества файлов
        train_count = len(os.listdir(os.path.join(fold_dir, "train", "images")))
        val_count = len(os.listdir(os.path.join(fold_dir, "valid", "images")))
        print(f"Фолд {fold}: Train={train_count}, Valid={val_count}")

        # Создаем YAML файл
        self._create_yaml_file(fold, fold_dir)

    def _copy_image_and_label(self, img_info, fold_dir, subset):
        """
        Копирует изображение и соответствующую метку в указанную директорию.

        Args:
            img_info (dict): Информация об изображении
            fold_dir (str): Путь к директории фолда
            subset (str): Подмножество данных ('train' или 'valid')
        """
        try:
            shutil.copy(
                img_info['image_path'],
                os.path.join(fold_dir, subset, "images")
            )
            shutil.copy(
                img_info['label_path'],
                os.path.join(fold_dir, subset, "labels")
            )
        except Exception as e:
            print(f"Ошибка копирования {img_info['name']}: {e}")

    def _create_yaml_file(self, fold, fold_dir):
        """
        Создает YAML-файл конфигурации для фолда.

        Args:
            fold (int): Номер фолда
            fold_dir (str): Путь к директории фолда
        """
        yaml_data = {
            'names': self.class_names,
            'nc': self.num_classes,
            'train': os.path.join(fold_dir, "train"),
            'val': os.path.join(fold_dir, "valid")
        }

        yaml_path = os.path.join(fold_dir, f"data_fold{fold}.yaml")
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_data, f, sort_keys=False)

        print(f"Создан YAML файл: {yaml_path}")

    def run_cross_validation(self):
        """Запуск кросс-валидации на всех фолдах."""
        for fold in range(1, self.num_folds + 1):
            print(f"\n=== Обработка фолда {fold} ===")

            fold_dir = os.path.join(self.base_path, f"fold_{fold}")
            yaml_path = os.path.join(fold_dir, f"data_fold{fold}.yaml")

            if not os.path.exists(yaml_path):
                print(f"YAML-файл не найден: {yaml_path}")
                continue

            # Инициализация и обучение модели
            model = YOLO(self.model_name)
            results = model.train(
                data=yaml_path,
                device=self.device,
                epochs=self.epochs,
                imgsz=self.imgsz,
                batch=self.batch,
                project=os.path.join(self.base_path, "runs"),
                name=f"fold_{fold}",
                exist_ok=True,
                verbose=True
            )

            # Валидация модели
            metrics = model.val(
                data=yaml_path,
                split='val',
                conf=0.5,
                iou=0.5,
                plots=True
            )

            # Сохранение метрик
            self._save_metrics(fold, metrics)

            # Визуализация PR-кривых
            self._plot_pr_curve(fold, metrics)

    def _save_metrics(self, fold, metrics):
        """
        Сохраняет метрики для текущего фолда.

        Args:
            fold (int): Номер фолда
            metrics: Объект с метриками от YOLO
        """
        fold_metrics = {
            'fold': fold,
            'map50': metrics.box.map50,
            'map': metrics.box.map,
            'precision': metrics.box.mean_results()[0],
            'recall': metrics.box.mean_results()[1],
        }

        if hasattr(metrics.box, 'maps') and metrics.box.maps is not None:
            for i, class_name in enumerate(self.class_names):
                fold_metrics[f'mAP_{class_name}'] = metrics.box.maps[i]
                self.class_metrics[class_name].append(metrics.box.maps[i])

        self.all_metrics.append(fold_metrics)

    def _plot_pr_curve(self, fold, metrics):
        """
        Строит и сохраняет Precision-Recall кривую для фолда.

        Args:
            fold (int): Номер фолда
            metrics: Объект с метриками от YOLO
        """
        try:
            plt.figure(figsize=(10, 6))
            for i, class_name in enumerate(self.class_names):
                p = metrics.box.class_result(i)[0]
                r = metrics.box.class_result(i)[1]
                plt.plot(
                    r, p,
                    label=f'{class_name} (AP={metrics.box.maps[i]:.2f})'
                )

            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.title(f'Precision-Recall Curve (Fold {fold})')
            plt.legend()
            plt.grid()
            plt.savefig(
                os.path.join(
                    self.base_path, "runs", f"pr_curve_fold{fold}.png"
                )
            )
            plt.close()
        except Exception as e:
            print(f"Ошибка при сохранении PR-кривой: {e}")

    def analyze_results(self):
        """Анализ и визуализация результатов кросс-валидации."""
        if not self.all_metrics:
            print("Нет данных для анализа!")
            return

        # Рассчитываем доверительные интервалы
        def confidence_interval(data, confidence=0.95):
            """Вычисляет среднее и доверительный интервал."""
            a = 1.0 * np.array(data)
            n = len(a)
            if n == 0:
                return 0, 0, 0
            mean, se = np.mean(a), np.std(a) / np.sqrt(n)
            h = se * 1.96  # Для 95% доверительного интервала
            return mean, mean - h, mean + h

        # Общие метрики
        map50_scores = [m['map50'] for m in self.all_metrics if 'map50' in m]
        mean_map50, low_map50, high_map50 = confidence_interval(map50_scores)

        map_scores = [m['map'] for m in self.all_metrics if 'map' in m]
        mean_map, low_map, high_map = confidence_interval(map_scores)

        print("\n=== Итоговые метрики ===")
        print(
            f"mAP@0.5: {mean_map50:.4f} "
            f"(95% ДИ: [{low_map50:.4f}, {high_map50:.4f}])"
        )
        print(
            f"mAP@0.5:0.95: {mean_map:.4f} "
            f"(95% ДИ: [{low_map:.4f}, {high_map:.4f}])"
        )

        # Метрики по классам
        print("\n=== Метрики по классам ===")
        for class_name in self.class_names:
            if class_name in self.class_metrics and self.class_metrics[class_name]:
                mean, low, high = confidence_interval(self.class_metrics[class_name])
                print(
                    f"{class_name}: {mean:.4f} "
                    f"(95% ДИ: [{low:.4f}, {high:.4f}])"
                )

        # Визуализация результатов
        self._plot_results(map50_scores, mean_map50, low_map50, high_map50)

        # Сохранение метрик
        self._save_metrics_to_csv()

    def _plot_results(self, map50_scores, mean_map50, low_map50, high_map50):
        """
        Визуализирует результаты кросс-валидации.

        Args:
            map50_scores (list): Значения mAP@0.5 для каждого фолда
            mean_map50 (float): Среднее значение mAP@0.5
            low_map50 (float): Нижняя граница доверительного интервала
            high_map50 (float): Верхняя граница доверительного интервала
        """
        plt.figure(figsize=(15, 6))

        # График mAP по фолдам
        plt.subplot(1, 2, 1)
        plt.bar(range(1, self.num_folds+1), map50_scores)
        plt.axhline(mean_map50, color='r', linestyle='--', label='Среднее')
        plt.fill_between(
            range(1, self.num_folds+1),
            low_map50,
            high_map50,
            alpha=0.2
        )
        plt.title('mAP@0.5 по фолдам')
        plt.xlabel('Фолд')
        plt.ylabel('mAP@0.5')
        plt.legend()

        # График mAP по классам
        plt.subplot(1, 2, 2)
        class_means = [
            np.mean(self.class_metrics[name]) if self.class_metrics[name] else 0
            for name in self.class_names
        ]
        plt.bar(self.class_names, class_means)
        plt.title('Средний mAP@0.5 по классам')
        plt.xlabel('Класс')
        plt.ylabel('mAP@0.5')
        plt.xticks(rotation=45)

        plt.tight_layout()
        plt.savefig(
            os.path.join(
                self.base_path, "runs", "cross_validation_results.png"
            )
        )
        plt.show()

    def _save_metrics_to_csv(self):
        """Сохраняет все метрики в CSV-файлы."""
        # Сохранение метрик по фолдам
        metrics_df = pd.DataFrame(self.all_metrics)
        metrics_df.to_csv(
            os.path.join(self.base_path, "runs", "all_metrics.csv"),
            index=False
        )

        # Сохранение метрик по классам
        class_metrics_df = pd.DataFrame(self.class_metrics)
        class_metrics_df.to_csv(
            os.path.join(self.base_path, "runs", "class_metrics.csv"),
            index=False
        )

        # Сохранение средних метрик
        mean_metrics = {
            'map50_mean': metrics_df['map50'].mean(),
            'map_mean': metrics_df['map'].mean(),
            'precision_mean': metrics_df['precision'].mean(),
            'recall_mean': metrics_df['recall'].mean(),
        }

        for class_name in self.class_names:
            mean_metrics[f'mAP_{class_name}_mean'] = np.mean(
                self.class_metrics[class_name]
            )

        pd.DataFrame([mean_metrics]).to_csv(
            os.path.join(self.base_path, "runs", "mean_metrics.csv"),
            index=False
        )

        print(
            "\nАнализ завершен. Результаты сохранены в:",
            os.path.join(self.base_path, "runs")
        )


if __name__ == "__main__":
    # Запуст кроссвалидации 
    cv = YOLOv8CrossValidation(
        base_path="/kaggle/working/kpk_dataset-9",
        num_folds=5,
        model_name="yolov8n.pt",
        epochs=50,
        imgsz=640,
        batch=32,
        device=0
    )

    # Подготовка данных
    cv.prepare_kfold_data()

    # Запуск кросс-валидации
    cv.run_cross_validation()

    # Анализ результатов
    cv.analyze_results()